In [67]:
import pandas as pd
import numpy as np

from pandas import get_dummies
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

import plotly.express as px




## 1.1 Leser filer

In [68]:
df_hospital=pd.read_csv('sample_data/hospital.csv')
df_physiological=pd.read_table('sample_data/physiological.txt')
df_demographic=pd.read_csv('sample_data/demographic.csv')

json_df=pd.read_json('sample_data/severity.json')
df_severity = json_df.iloc[:, 0:-1].explode(list(json_df.columns[2:-1])) 
df_severity.reset_index(drop=True, inplace=True)


#### 1.1.1 Slår sammen tabellene

In [69]:
df=df_demographic.merge(df_hospital,on='pasient_id',how='outer')\
    .merge(df_physiological,on='pasient_id',how='outer')\
    .merge(df_severity,on='pasient_id',how='outer')

## 1.2 Fjerner 

In [70]:
df = df.drop_duplicates(subset=['pasient_id']).reset_index(drop=True)
df.set_index('pasient_id', inplace=True)

In [71]:
df.drop(columns=['sykehusdød'], inplace=True)

df.drop(columns=['dødsfall'], inplace=True)


In [72]:
df.drop(columns=['adl_pasient'], inplace=True)


df.drop(columns=['bilirubin'], inplace=True)


In [73]:
df.drop(columns=['dnr_status'], inplace=True)


In [74]:
df.drop(columns=['sykdomskategori_id'], inplace=True)

#### 1.2.1 Setter negative verdier for alder til NaN

In [75]:
df['alder'] = df['alder'].apply(lambda x: x if pd.isnull(x) or x > 0 else np.nan)


## 1.3.1 Kategorisk til numerisk 

#### 1.3.1 Kjønn og etnisitet

In [76]:
most_frequent_etnisitet = df['etnisitet'].mode()[0] 

df['etnisitet'] = df['etnisitet'].fillna(most_frequent_etnisitet)

etnisitet_mapping = {
    'white': 0,
    'black': 1,
    'asian': 2,
    'hispanic': 3,
    'other': 4,
}

df['etnisitet'] = df['etnisitet'].map(etnisitet_mapping)


In [77]:
df=get_dummies(df, columns=['kjønn'])
df['kjønn'] = df['kjønn_male'].astype(int)  
df.drop(columns=['kjønn_female', 'kjønn_male'], inplace=True) 


#### 1.3.2 Sykehuskategori, underkategori og kreft.

In [78]:
sykdomskategori_mapping = {
            'ARF/MOSF': 4,
            'COPD/CHF/Cirrhosis': 1,
            'Coma': 3,
            'Cancer': 2,
    }
df['sykdomskategori'] = df['sykdomskategori'].map(sykdomskategori_mapping)


In [79]:
columns_to_encode = ['kreft', 'sykdom_underkategori']

encoder = OneHotEncoder(drop=None, sparse_output=False)

df_encoded_columns = encoder.fit_transform(df[columns_to_encode])
df_encoded_df = pd.DataFrame(df_encoded_columns, columns=encoder.get_feature_names_out(columns_to_encode))

df = df.drop(columns=columns_to_encode).reset_index(drop=True)
df_encoded_df = df_encoded_df.reset_index(drop=True)
df = pd.concat([df, df_encoded_df], axis=1)



In [80]:
df['kreft'] = ((df['kreft_yes'] == 1) | (df['kreft_metastatic'] == 1)).astype(int)
df.drop(columns=['kreft_yes', 'kreft_no'], inplace=True)

#### 1.3.3 Inntekt og utdanning

In [81]:
income_mapping= {
    'under $11k': 0,
    '$11-$25k': 1,
    '$25-$50k': 2,
    '>$50k': 3,
}

df['inntekt'] = df['inntekt'].map(income_mapping).astype(float)

In [82]:
bins = [1, 11, 14, 17, float('inf')]
labels = ['grunnskole/ungdomskole', 'videregående', 'høyere utdanning', 'postgraduate']

df['utdanning_kategori'] = pd.cut(df['utdanning'], bins=bins, labels=labels, include_lowest=True)

utdanning_mapping = {
    'grunnskole/ungdomskole': 0,
    'videregående': 1,
    'høyere utdanning': 2,
    'videre utdanning': 3,
}

df['utdanning_kategori'] = df['utdanning_kategori'].map(utdanning_mapping)

df.drop(columns=['utdanning'], inplace=True)


## 1.4.1 Gjør om navn og konverterer

In [83]:
df.rename(columns={
    'sykdom_underkategori_Coma': 'sykdom_Coma',
    'sykdom_underkategori_Cancer': 'sykdom_Cancer',
    'sykdom_underkategori_Cirrhosis': 'sykdom_Cirrhosis',
    'sykdom_underkategori_ARF/MOSF w/Sepsis': 'sykdom_ARF/MOSF w/Sepsis',
    'sykdom_underkategori_COPD': 'sykdom_COPD',
    'sykdom_underkategori_CHF': 'sykdom_CHF',
    'sykdom_underkategori_Lung Cancer': 'sykdom_Lung_Cancer',
    'sykdom_underkategori_Colon Cancer': 'sykdom_Colon_Cancer',
    'sykdom_underkategori_MOSF w/Malig': 'sykdom_MOSF w/Malig'
}, inplace=True)

In [84]:
columns_to_convert=[ 
    'antall_komorbiditeter', 'adl_stedfortreder' , 'fysiologisk_score' , 'apache_fysiologisk_score',
    'overlevelsesestimat_2mnd', 'overlevelsesestimat_6mnd', 'diabetes', 'demens', 'lege_overlevelsesestimat_2mnd',
    'lege_overlevelsesestimat_6mnd', 'utdanning_kategori'
]
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')


## 2.1.0 Fyller inn for NaN

In [85]:
df.loc[:, 'blod_ph'] = df['blod_ph'].fillna(7.40)

In [86]:
df.loc[:, 'lungefunksjon'] = df['lungefunksjon'].fillna(333.3)

In [87]:
df.loc[:, 'blodurea_nitrogen'] = df['blodurea_nitrogen'].fillna(6.51)

In [88]:
df.loc[:, 'glukose'] = df['glukose'].fillna(85)


In [89]:
df.loc[:, 'urinmengde'] = df['urinmengde'].fillna(2502)


In [90]:
imputer = SimpleImputer(strategy='mean')
df[:] = imputer.fit_transform(df)
df.isnull().sum()

alder                            0
inntekt                          0
etnisitet                        0
blodtrykk                        0
hvite_blodlegemer                0
hjertefrekvens                   0
respirasjonsfrekvens             0
kroppstemperatur                 0
lungefunksjon                    0
serumalbumin                     0
kreatinin                        0
natrium                          0
blod_ph                          0
glukose                          0
blodurea_nitrogen                0
urinmengde                       0
sykdomskategori                  0
antall_komorbiditeter            0
koma_score                       0
adl_stedfortreder                0
fysiologisk_score                0
apache_fysiologisk_score         0
overlevelsesestimat_2mnd         0
overlevelsesestimat_6mnd         0
diabetes                         0
demens                           0
lege_overlevelsesestimat_2mnd    0
lege_overlevelsesestimat_6mnd    0
kjønn               

## 3.1 Variabelutvinning

In [91]:
df['fysiologisk_score_merge'] = (df['fysiologisk_score'] + df['apache_fysiologisk_score']) / 2
df.drop(columns=['fysiologisk_score', 'apache_fysiologisk_score'], inplace=True)

In [92]:
df['fysiologisk_komorbiditet_mult'] = df['fysiologisk_score_merge'] * df['antall_komorbiditeter']

In [93]:
df['nyrefunksjon'] = df[['blodurea_nitrogen', 'kreatinin', 'natrium']].mean(axis=1)
df.drop(columns=['blodurea_nitrogen', 'kreatinin', 'natrium'], inplace=True)

In [94]:
df['hjertefunksjon'] = df[['blodtrykk', 'hjertefrekvens']].mean(axis=1)
df.drop(columns=['blodtrykk', 'hjertefrekvens'], inplace=True)


In [95]:
df['kreft_sammenslått'] = df[['kreft', 'kreft_metastatic']].max(axis=1)
df.drop(columns=['kreft', 'kreft_metastatic'], inplace=True)

#### 3.1.1 Fjerner kolonner

In [96]:
df.drop(columns=[
    'sykdom_Lung_Cancer',
    'sykdom_Colon_Cancer',
    'sykdom_ARF/MOSF w/Sepsis', 
    'sykdom_CHF', 
    'sykdom_COPD', 
    'sykdom_Cirrhosis', 
    'sykdom_Coma', 
    'sykdom_MOSF w/Malig',
    ], inplace=True)

In [97]:
import pickle

with open('model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

In [98]:
predicted_model= model.predict(df)
pd.DataFrame(predicted_model, columns=['predicted']).to_csv('predictions.csv', index=False)

In [99]:
predictions_df = pd.read_csv('predictions.csv')

fig = px.histogram(predictions_df, x='predicted', title='Predikert sykehusopphold')
fig.show()